# Setup
## Colab: Install packages (must restart runtime after)

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
%env NCCL_P2P_DISABLE=1
%env NCCL_IB_DISABLE=1

GLOBALS = {"colab": "google.colab" in str(get_ipython())}

if GLOBALS["colab"]:
    ! pip install transformers -q
    ! pip install datasets accelerate peft -q
    ! pip install accelerate -qU
    ! pip install wandb -qU
    # ! pip install pysentimiento -q

    from google.colab import drive, userdata

## Imports

In [ ]:
import itertools
import os
import sys

import pandas as pd
import torch

if GLOBALS["colab"]:
    drive.mount("/content/drive")
    # Clone repo
    ! git clone https://github.com/Pastells/Human-vs.-Machine-Perceptions-on-Immigration-Stereotypes.git

    # Create results and models folders in /content
    ! mkdir -p results/metrics
    ! mkdir -p results/trainer_logs
    ! mkdir -p models/stereohoax

    sys.path.append("/content/Human-vs.-Machine-Perceptions-on-Immigration-Stereotypes")
    os.environ["WANDB_API_KEY"] = userdata.get("wandb")
else:
    secret = pd.read_csv("secret.config", header=None)
    os.environ["WANDB_API_KEY"] = secret[1][0]

In [ ]:
from utils.fine_tuning import main_loop

pd.set_option("display.max_colwidth", None)
pd.set_option("display.precision", 3)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["WANDB_LOG_MODEL"] = "false"
os.environ["WANDB_DISABLED"] = "false"

# Inputs

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GLOBALS["experiment_name"] = "proves"
GLOBALS["max_tokens"] = 128
GLOBALS["device"] = DEVICE
# -----------------------------------
# Technical variables, single values
# -----------------------------------

H_PARAMS = {
    "output_dir": "/content" if GLOBALS["colab"] else ".",
    "save_models": False,  # Save models to Drive
    "batch_size": 32,
    "epochs": 10,  # 20,
    "n_steps": 50,
    "patience": 8,  # times N_STEPS
    "class_weights": None,
    "save": 5,  # How many checkpoints to keep
    "lora": False,  # LoRA if True, FFT otherwise
    "fp16": False,
}
# H_PARAMS["n_steps"] = H_PARAMS["n_steps"] / H_PARAMS["batch_size"] * 32

# Layer-Wise Layer Rate Decay
LLRD = {
    "LLRD": False,  # {False, True, "grouped"}
    "lr_decay": 0.9,  # if LLRD is True
    "C1": 1.75,  # if LLRD == grouped
    "C2": 3.5,  # if LLRD == grouped
}

DATA_PARAMS = {
    "append": False,  # Append context instead of using [SEP] token
    "fill": True,  # Fill missing contexts
    "cont_same_as_txt": True,
    "fill_same": True,
    "files": None,  # Change if you want custom files (see cell below for few-shot)
}

# ------------------------------------
# SETUP
# If multiple values -> loop
# ------------------------------------
# Do not mix different data and context, each data has its own context
SEEDS = (42,)
DATA_SOFT = (
    ("stereohoax", False),
    ("stereohoax", True),
)
SEEDS_DATA_SOFT = itertools.product(SEEDS, DATA_SOFT)

MODELS = {
    # "beto": "dccuchile/bert-base-spanish-wwm-cased",
    "roberta_bne": "PlanTL-GOB-ES/roberta-base-bne",
    # "roberta_large_bne": "PlanTL-GOB-ES/roberta-large-bne",
    # "bertin": "bertin-project/bertin-roberta-base-spanish",
    # "mbert": "google-bert/bert-base-multilingual-cased",
    # "robertuito": "pysentimiento/robertuito-base-uncased", # raises CUDA error for max_tokens > 128
    # "albeto": "dccuchile/albert-base-spanish",
    # "beto_uncased": "dccuchile/bert-base-spanish-wwm-uncased",
}

CONTEXTS = (None,)

# Models loop (main)

In [ ]:
main_loop(SEEDS_DATA_SOFT, DATA_PARAMS, CONTEXTS, H_PARAMS, MODELS, GLOBALS, LLRD)